# Sistemas de Recomendação baseado em contexto

Neste trabalho será apresentado um exemplo de sistema de recomendação baseado em contexto, mais especificamente usando o contexto de informação social. 

As técnicas de sistemas de recomendação baseado em contexto são divididas em 3 categorias: Pré-filtro contextual, Pós-filtro contextual e modelagem contextual. Usaremos pra este sistema o Pré-filtro.

Vamos começar carregando o nosso dataset. O dataset utilizado é um disponibilizado pelo IMDB.

In [1]:
# Import Pandas
import pandas as pd

# Load Movies Metadata
metadata = pd.read_csv('./movie_dataset/movies_metadata.csv', low_memory=False)

# Print the first three rows
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [2]:
metadata.shape

(45466, 24)

Observe que o nosso dataset é muito grande, tem no total de 45466 filmes, para ser testado.
Por isso iremos diminui-lo. Para fazer isso vamos criar um novo metadata que tenha uma quantidade mínima de avaliações.
No nosso sistema vamos fazer com que apenas os top 10% dos filmes entraram no novo dataset

In [3]:
m = metadata['vote_count'].quantile(0.90)
print(m)

160.0


In [4]:
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

(4555, 24)

O nosso novo dataset tem no total de 4555 filmes.

In [5]:
q_movies = q_movies.reset_index(drop=True)
q_movies.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
3,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
4,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
5,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0
6,False,NaN,62000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,9087,tt0112346,en,The American President,"Widowed U.S. president Andrew Shepherd, one of...",...,1995-11-17,107879496.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Why can't the most powerful man in the world h...,The American President,False,6.5,199.0
7,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,12110,tt0112896,en,Dracula: Dead and Loving It,When a lawyer shows up at the vampire's doorst...,...,1995-12-22,0.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Dracula: Dead and Loving It,False,5.7,210.0
8,False,"{'id': 117693, 'name': 'Balto Collection', 'po...",0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",NaN,21032,tt0112453,en,Balto,An outcast half-wolf risks his life to prevent...,...,1995-12-22,11348324.0,78.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Part Dog. Part Wolf. All Hero.,Balto,False,7.1,423.0
9,False,NaN,52000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,524,tt0112641,en,Casino,The life of the gambling paradise – Las Vegas ...,...,1995-11-22,116112375.0,178.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,No one stays at the top forever.,Casino,False,7.8,1343.0


Agora temos o nosso dataset pronto para ser utilizado no sistema.

## Pré-Filtro

Agora vamos realizar o pré-filtro, para fazer isso vamos utilizar uma informação social. No nosso dataset temos uma coluna de gênero, com ela podemos encontrar a melhor opção de filmes para alguns círculos social do usuário, por exemplo usar o gênero 'Family' para ver com a família ou o gênero 'Romance' para ver com o/a namorado/a.

O pré-filtro vai realizar uma filtragem nos dados onde apenas será necessário observar os dados de um determinado contexto.

No nosso exemplo vamos querer um filme pra ver com a família.

In [6]:
social_information = 'Family'

In [7]:
q_movies = q_movies[q_movies.genres.str.contains(social_information) == True]
q_movies.shape

(541, 24)

In [8]:
q_movies = q_movies.reset_index(drop=True)
q_movies.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 117693, 'name': 'Balto Collection', 'po...",0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",NaN,21032,tt0112453,en,Balto,An outcast half-wolf risks his life to prevent...,...,1995-12-22,11348324.0,78.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Part Dog. Part Wolf. All Hero.,Balto,False,7.1,423.0
3,False,"{'id': 9435, 'name': 'Babe Collection', 'poste...",30000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 18, 'na...",NaN,9598,tt0112431,en,Babe,Babe is a little pig who doesn't quite know hi...,...,1995-07-18,254134910.0,89.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A little pig goes a long way.,Babe,False,6.0,756.0
4,False,"{'id': 136214, 'name': 'Pocahontas Collection'...",55000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",NaN,10530,tt0114148,en,Pocahontas,History comes gloriously to life in Disney's e...,...,1995-06-14,346079773.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,An American legend comes to life.,Pocahontas,False,6.7,1509.0
5,False,"{'id': 8819, 'name': 'Casper Collection', 'pos...",50000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 35, 'na...",NaN,8839,tt0112642,en,Casper,Furious that her late father only willed her h...,...,1995-05-26,287928194.0,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Get an afterlife,Casper,False,6.0,1045.0
6,False,"{'id': 410261, 'name': 'A Goofy Movie Collecti...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 16, ...",NaN,15789,tt0113198,en,A Goofy Movie,"Though Goofy always means well, his amiable cl...",...,1995-04-07,35348597.0,78.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It's the story of a father who couldn't be clo...,A Goofy Movie,False,6.7,404.0
7,False,NaN,60000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...",NaN,6280,tt0110216,en,Junior,"As part of a fertility research project, a mal...",...,1994-11-22,37000000.0,109.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Nothing is inconceivable,Junior,False,4.7,381.0
8,False,NaN,17000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",NaN,19101,tt0113670,en,A Little Princess,When her father enlists to fight for the Briti...,...,1995-05-10,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Every girl everywhere is a princess.,A Little Princess,False,7.4,207.0
9,False,NaN,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 35, '...",NaN,11008,tt0110443,en,Major Payne,Major Benson Winifred Payne is being discharge...,...,1995-03-24,0.0,95.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,He's looking for a few good men... or a few gu...,Major Payne,False,6.1,164.0


Com o pré-filtro, o nosso dataset deixou de ter 4555 filmes para ter 541 filmes recomendados para a família.

O recomendador sensível ao contexto acabou de fazer a sua tarefa, agora um novo recomendador deverá ser utilizado para fazer a recomendação. No nosso exemplo vamos utilizar um recomendador de baseado em conteúdo.

## Recomendador baseado em conteúdo

### Recomendador baseado em créditos, gêneros e palavras-chave.

Nesta seção, vamos construir um recomendador baseado nos seguintes metadata: os top3 atores, o diretor, gêneros relacionados e as palavras-chave dos filmes.

Os dados das palavras-chave, do elenco e da equipe não estão disponíveis no nosso atual dataset, então o nosso primeiro passo vai ser carregar e juntar eles no nosso dataframe principal.

In [9]:
# Load keywords and credits
credits = pd.read_csv('./movie_dataset/credits.csv')
keywords = pd.read_csv('./movie_dataset/keywords.csv')

# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
q_movies['id'] = q_movies['id'].astype('int')

# Merge keywords and credits into your main q_movies dataframe
q_movies = q_movies.merge(credits, on='id')
q_movies = q_movies.merge(keywords, on='id')

In [10]:
q_movies.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 117693, 'name': 'Balto Collection', 'po...",0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",NaN,21032,tt0112453,en,Balto,An outcast half-wolf risks his life to prevent...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Part Dog. Part Wolf. All Hero.,Balto,False,7.1,423.0,"[{'cast_id': 1, 'character': 'Balto (voice)', ...","[{'credit_id': '593f24b9c3a3680369002371', 'de...","[{'id': 1994, 'name': 'wolf'}, {'id': 6411, 'n..."
3,False,"{'id': 9435, 'name': 'Babe Collection', 'poste...",30000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 18, 'na...",NaN,9598,tt0112431,en,Babe,Babe is a little pig who doesn't quite know hi...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A little pig goes a long way.,Babe,False,6.0,756.0,"[{'cast_id': 1, 'character': 'Babe the Gallant...","[{'credit_id': '52fe450fc3a36847f80b9ff7', 'de...","[{'id': 3020, 'name': 'sheep'}, {'id': 4931, '..."
4,False,"{'id': 136214, 'name': 'Pocahontas Collection'...",55000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",NaN,10530,tt0114148,en,Pocahontas,History comes gloriously to life in Disney's e...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,An American legend comes to life.,Pocahontas,False,6.7,1509.0,"[{'cast_id': 1, 'character': 'Pocahontas (voic...","[{'credit_id': '52fe43819251416c75013087', 'de...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."


Vamos computar vetores de Term Frequency-Inverse Document Frequency(TF-IDF) para cada documento. Com isso vamos ter uma matriz onde cada coluna representa uma palavra no vocabulário do overview, que são todas as palavras que aparecem pelo menos uma vez, e cada coluna representa um filme.

Na essência a nota TF-IDF é a frequência das palavras que aparecem no documento, o peso de uma palavra vai diminuir de acordo com a quantidade de vezes que ela aparece no documento. Isso é feito para reduzir a importância de palavras que aparecem com uma certa frequência no overview.

Felizmente, scikit-learn tem uma classe TfIdfVectorizer que produz a matriz TF-IDF.

In [12]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
q_movies['overview'] = q_movies['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(q_movies['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(543, 5807)

A partir disso, podemos ver que mais de 5800 de palavras diferentes foram utilizadas para descrever os mais de 540 filmes no dataset.

Com essa matriz, podemos calcular uma nota de similaridade. Temos muitas maneiras de calcular a nota, como pela escala de distância euclidiana, pela escala de correlação de Pearson e pela similaridade do cosseno.

Neste documento, vamos usar a similaridade do cosseno para calcular a quantidade numérica que mostra a similaridade entre dois filmes. A similaridade do cosseno é definido como:
$$cosine \left(x,y\right) = \frac{x.y^T}{||x||.||y||}$$
Já que nós utilizamos o TF-IDF vectorizer, calcular o produto escalar vai diretamente nos dar a nota da similaridade do cosseno.
Portanto, vamos usar <b>linear_kernel()</b> do <b>sklearn</b> ao invés da similaridade do cosseno já que é mais rápido.

In [13]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

Agora que temos uma tfidf_matrix menor, foi possível executar o código acima.

Vamos criar uma função que utiliza o nome de um filme como entrada e retorna uma lista dos 10 filmes mais parecidos. Para isso vamos precisar de um mecanismo que identifique o indíce de um filme no dataframe, dado o seu título.

In [14]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(q_movies.index, index=q_movies['title']).drop_duplicates()

Agora estamos prontos para criar nossa função de recomendação.

In [16]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return q_movies['title'].iloc[movie_indices]

Com as nossas novas características, elenco, equipe e palavras-chave, vamos extrair os 3 mais importantes atores, o diretor e as palavras-chave associadas com aquele filme. Temos que converter essas características de uma forma que seja útil para nós

In [17]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    q_movies[feature] = q_movies[feature].apply(literal_eval)

Agora vamos escrever funções que vão ajudar na extração das informações desejadas de cada característica. Primeiro, vamos importar o pacote NumPy para ter acesso às constantes NaN. Em seguida, podemos usá-las para escrever a função <b>get_director</b>().

In [18]:
# Import Numpy 
import numpy as np

In [19]:
# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [20]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [21]:
# Define new director, cast, genres and keywords features that are in a suitable form.
q_movies['director'] = q_movies['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    q_movies[feature] = q_movies[feature].apply(get_list)

In [22]:
# Print the new features of the first 3 films
q_movies[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Balto,"[Kevin Bacon, Bob Hoskins, Bridget Fonda]",Simon Wells,"[wolf, dog-sledding race, alaska]","[Family, Animation, Adventure]"


O próximo passo será coverter os nomes e instâncias de palavra-chave para minúsculo e retirar todos os espaços entre eles. Isto é feito para que seu vetorizador não considere o Johnny de "Johnny Depp" e "Johnny Galecki" como os mesmos. Depois desse passo, os atores citados acima serão representados como "johnnydepp" e "johnnygalecki" e vão ser diferentes no vetorizador.

In [23]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [24]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    q_movies[feature] = q_movies[feature].apply(clean_data)

Agora podemos criar "metadata soup", no qual é uma string que contém todos os metadata que nós queremos para alimentar nosso vetorizador(chamados de atores, diretores e palavras-chave).

In [25]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [26]:
# Create a new soup feature
q_movies['soup'] = q_movies.apply(create_soup, axis=1)

Os próximos passos são os mesmo que a gente fez no recomendador baseado na descrição da sinopse. Uma diferença importante é que vamos usar <b>CountVectorizer()</b> ao invés de TF-IDF. Isso é porque não importa se tem atores que atuaram em mais filmes ou diretores que dirigiram mais filmes.

In [27]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(q_movies['soup'])

In [28]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [29]:
# Reset index of your main DataFrame and construct reverse mapping as before
q_movies = q_movies.reset_index()
indices = pd.Series(q_movies.index, index=q_movies['title'])

Agora podemos reutilizar nossa função <b>get_recommendations()</b> passando na nova matriz <b>cosine_sim2</b> como seu segundo argumento.

In [30]:
get_recommendations('Toy Story', cosine_sim2)

119                             Toy Story 2
355                             Toy Story 3
467              Toy Story That Time Forgot
436                    Toy Story of Terror!
263                           Monster House
381                                  Cars 2
537    Lilo & Stitch 2: Stitch has a Glitch
53                       The Wrong Trousers
214               Christmas with the Kranks
484     Barbie as The Princess & the Pauper
Name: title, dtype: object

In [32]:
get_recommendations('Jumanji', cosine_sim2)

335            Where the Wild Things Are
23                        The Pagemaster
459    Tinker Bell and the Lost Treasure
308                        City of Ember
294                      The Water Horse
427                                 Epic
38                      The Wizard of Oz
50       Aladdin and the King of Thieves
54                    The Princess Bride
71                             Labyrinth
Name: title, dtype: object